In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import kagglehub
import os

## Load Feature Group from Hopsworks

In [2]:
import hopsworks 
project = hopsworks.login()
fs = project.get_feature_store(name='recipe_recommender_featurestore')
review_fg = fs.get_feature_group('review', version=1)
recipe_fg = fs.get_feature_group('recipe', version=1)

2024-12-10 17:06:14,286 INFO: Initializing external client
2024-12-10 17:06:14,286 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-10 17:06:20,836 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1193139


In [4]:
train_recipe_data = recipe_fg.read()

# Display the DataFrame
print(train_recipe_data.info())

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (417.42s) 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418013 entries, 0 to 418012
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   recipeid             418013 non-null  int64  
 1   name                 418013 non-null  object 
 2   authorid             418013 non-null  int64  
 3   totaltime            418013 non-null  float64
 4   recipecategory       418013 non-null  object 
 5   aggregatedrating     418013 non-null  float64
 6   reviewcount          418013 non-null  float64
 7   calories             418013 non-null  float64
 8   fatcontent           418013 non-null  float64
 9   saturatedfatcontent  418013 non-null  float64
 10  cholesterolcontent   418013 non-null  float64
 11  sodiumcontent        418013 non-null  float64
 12  carbohydratecontent  418013 non-null  float64
 13  fibercontent         418013 non

In [87]:
train_review_data = review_fg.read()

print(train_review_data.info())

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (221.61s) 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   reviewid   500000 non-null  int64  
 1   recipeid   500000 non-null  int64  
 2   authorid   500000 non-null  int64  
 3   rating     500000 non-null  float64
 4   review     500000 non-null  object 
 5   year       500000 non-null  float64
 6   month_sin  500000 non-null  float64
 7   month_cos  500000 non-null  float64
dtypes: float64(4), int64(3), object(1)
memory usage: 30.5+ MB
None


In [ ]:
train_recipe_data['Name'] = list(name_embeddings)
train_recipe_data['RecipeCategory'] = list(category_embeddings)
train_review_data['Review'] = list(review_embeddings)

In [5]:
train_recipe_data.head()

,recipeid,name,authorid,totaltime,recipecategory,aggregatedrating,reviewcount,calories,fatcontent,saturatedfatcontent,cholesterolcontent,sodiumcontent,carbohydratecontent,fibercontent,sugarcontent,proteincontent
0,426676,"[-0.09981661, 0.057647053, -0.0117327105, 0.04...",388463,-0.023333,"[-0.046547513, 0.093352586, 0.050087087, 0.047...",-0.946300,-0.182754,-0.151172,-0.060352,0.017690,0.134915,-0.111967,-0.167798,-0.313982,-0.125558,-0.195473
1,2130,"[-0.07288596, -0.062904075, -0.008518769, 0.05...",1534,-0.027823,"[0.005264935, 0.007629845, 0.0011869151, 0.039...",0.776211,-0.049050,-0.232925,-0.128901,-0.042847,-0.310359,-0.169850,-0.181529,-0.237037,-0.085174,-0.381872
2,515573,"[-0.031779263, -0.05110866, -0.014522817, -0.0...",226863,-0.023333,"[-0.046547513, 0.093352586, 0.050087087, 0.047...",1.206839,-0.115902,0.003697,-0.016285,-0.091667,-0.053098,-0.096693,-0.046251,0.246617,-0.101199,0.342488
3,505501,"[-0.07791573, 0.021957776, -0.059152737, 0.055...",220195,-0.033436,"[-0.045499124, 0.055375434, -0.021843556, -0.0...",-0.946300,-0.182754,-0.116163,-0.141958,-0.167826,-0.333958,6.258109,0.075804,1.158966,0.052002,-0.176597
4,162817,"[-0.061670627, -0.023237374, 0.027469449, -0.0...",80353,-0.034109,"[-0.01266546, -0.04918345, 0.08920136, 0.00925...",1.206839,0.218359,-0.044828,-0.201530,-0.187354,-0.333958,-0.168596,0.298045,-0.423904,0.543017,-0.410185


In [8]:
train_review_data.head()

,reviewid,recipeid,authorid,rating,review,year,month_sin,month_cos
0,376275,201965,424680,0.465609,"[-0.03471386, -0.0034809876, 0.025403176, 0.01...",-0.522026,0.866025,5.000000e-01
1,565527,203782,667915,0.465609,"[-0.048685435, -0.05063994, 0.0123515455, 0.07...",-0.252375,0.500000,8.660254e-01
2,277879,36123,176615,0.465609,"[-0.10000747, -0.06447308, -0.005737721, 0.062...",-0.791677,0.500000,-8.660254e-01
3,746144,19177,52648,-0.320427,"[-0.08373337, 0.005466032, -0.02001648, 0.0557...",-0.252375,-0.500000,8.660254e-01
4,1253618,255645,424680,0.465609,"[-0.06095889, -0.015254693, 0.03496923, -0.011...",0.556577,-1.000000,-1.836970e-16


In [212]:
train_review_data.shape

(397019, 8)

# Model Implementation

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset


## Review Model

### Review Dataset

In [12]:
class ReviewsDataset(Dataset):
    def __init__(self, reviews_df):
        """
        Args:
            reviews_df (pd.DataFrame): Dataframe containing the reviews data.
        """
        self.reviews_df = reviews_df

    def __len__(self):
        # Return the total number of samples
        return len(self.reviews_df)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the sample to fetch.
        Returns:
            A dictionary with tensors for the input features and target rating.
        """
        row = self.reviews_df.iloc[idx]
        
        # Extract features
        review_id = row["reviewid"]
        recipe_id = row["recipeid"]
        author_id = row["authorid"]
        review_embedding = torch.tensor(row["review"], dtype=torch.float32)  # Convert list to tensor
        year = torch.tensor(row["year"], dtype=torch.float32)
        month_sin = torch.tensor(row["month_sin"], dtype=torch.float32)
        month_cos = torch.tensor(row["month_cos"], dtype=torch.float32)
        # Rating (target)
        rating = torch.tensor(row["rating"], dtype=torch.float32)
        # concat_features = torch.cat(review_embedding, month_sin, month_cos)
        # print(concat_features.shape)
        return {
            "review_id": review_id,
            "recipe_id": recipe_id,
            "author_id": author_id,
            "review_embedding": review_embedding,
            "year": year,
            "month_sin": month_sin,
            "month_cos": month_cos,
            "rating": rating
        }

### Review Tower

In [20]:
class ReviewTower(nn.Module):
    def __init__(self, review_embedding_dim, embedding_dim):
        """
        Args:
            review_embedding_dim (int): Dimensionality of the review feature vector.
            embedding_dim (int): Dimensionality of the output embedding vector.
        """
        super(ReviewTower, self).__init__()
        self.fc1 = nn.Linear(review_embedding_dim + 2 + 1 + 1, 128)  # Add month_sin, month_cos, and year, and rating
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, embedding_dim)  # Final embedding layer

    def forward(self, review_embedding, month_sin, month_cos, year, rating):
        """
        Forward pass for the review tower.
        
        Args:
            review_embedding (Tensor): Review feature vector (e.g., sentence embedding).
            month_sin (Tensor): Sinusoidal month feature.
            month_cos (Tensor): Cosine month feature.
            year (Tensor): Year feature.
            rating (Tensor): Rating given to recipe
        
        Returns:
            Tensor: Embedded vector for the review.
        """
        if isinstance(review_embedding, pd.Series):
            review_embedding = torch.stack([torch.tensor(embed, dtype=torch.float32) for embed in review_embedding])
            month_sin = torch.tensor(month_sin.values, dtype=torch.float32)
            month_cos = torch.tensor(month_cos.values, dtype=torch.float32)
            year = torch.tensor(year.values, dtype=torch.float32)
            rating = torch.tensor(rating.values, dtype=torch.float32)
            
        # Concatenate all features (now all are 2D tensors)
        x = torch.cat((review_embedding, month_sin.unsqueeze(1), month_cos.unsqueeze(1), year.unsqueeze(1), rating.unsqueeze(1)), dim=1)
        
        # Pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Final embedding vector
        return x


## Recipe Model

### Recipe Dataset

In [14]:
class RecipeDataset(Dataset):
    def __init__(self, recipes_df):
        """
        Args:
            recipes_df (pd.DataFrame): Dataframe containing the reviews data.
        """
        self.recipes_df = recipes_df

    def __len__(self):
        # Return the total number of samples
        return len(self.recipes_df)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the sample to fetch.
        Returns:
            A dictionary with tensors for the input features and target rating.
        """
        row = self.recipes_df.iloc[idx]
        
        # Extract features
        recipe_id = row["recipeid"]
        name_embedding = torch.tensor(row["name"], dtype=torch.float32)
        author_id = row["authorid"]
        total_time = torch.tensor(row["totaltime"], dtype=torch.float32)
        category_embedding = torch.tensor(row["recipecategory"], dtype=torch.float32)
        aggr_rating = torch.tensor(row["aggregatedrating"], dtype=torch.float32)
        review_count = torch.tensor(row["reviewcount"], dtype=torch.float32)
        calories = torch.tensor(row["calories"], dtype=torch.float32)
        fat_content = torch.tensor(row["fatcontent"], dtype=torch.float32)
        saturated_fat_content = torch.tensor(row["saturatedfatcontent"], dtype=torch.float32)
        cholesterol_content = torch.tensor(row["cholesterolcontent"], dtype=torch.float32)
        sodium_content = torch.tensor(row["sodiumcontent"], dtype=torch.float32)
        carbohydrate_content = torch.tensor(row["carbohydratecontent"], dtype=torch.float32)
        fiber_content = torch.tensor(row["fibercontent"], dtype=torch.float32)
        sugar_content = torch.tensor(row["sugarcontent"], dtype=torch.float32)
        protein_content = torch.tensor(row["proteincontent"], dtype=torch.float32)

        return {
            "recipe_id": recipe_id,
            "name_embedding": name_embedding,
            "author_id": author_id,
            "total_time": total_time,
            "category_embedding": category_embedding,
            "aggr_rating": aggr_rating,
            "calories": calories,
            "review_count" : review_count,
            "fat_content": fat_content,
            "saturated_fat_content": saturated_fat_content,
            "cholesterol_content": cholesterol_content,
            "sodium_content" : sodium_content,
            "carbohydrate_content" : carbohydrate_content,
            "fiber_content" : fiber_content,
            "sugar_content" : sugar_content,
            "protein_content" : protein_content
        }

### Recipe Tower

In [101]:
class RecipeTower(nn.Module):
    def __init__(self, recipe_embedding_dim, embedding_dim):
        """
        Args:
            review_vector_dim (int): Dimensionality of the review feature vector.
            embedding_dim (int): Dimensionality of the output embedding vector.
        """
        super(RecipeTower, self).__init__()
        self.fc1 = nn.Linear(recipe_embedding_dim * 2 + 14, 128)  # Add month_sin, month_cos, and year, and rating
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, embedding_dim)  # Final embedding layer

    def forward(self, recipe_id, name_embedding, author_id, total_time, category_embedding,
                aggr_rating, review_count, calories, fat_content, saturated_fat_content,
                cholesterol_content, sodium_content, carbohydrate_content, fiber_content,
                sugar_content, protein_content
                ):
        """
        Forward pass for the recipe tower.
        
        Args:
            name_embedding (Tensor): Embedding made for the recipe name.
            author_id (Tensor): ID for the author.
            total_time (Tensor): Parsed total time taken to prepare the recipe.
            category_embedding (Tensor): Embedding made for the recipe category.
            aggr_rating (Tensor): Average rating for the recipe.
            review_count (Tensor): Count of reviews for the recipe.
            (The rest are very self-explanatory)
        
        Returns:
            Tensor: Embedded vector for the review.
        """
        if isinstance(recipe_id, pd.Series):
            recipe_id = torch.tensor(recipe_id.values, dtype=torch.float32)

            name_embedding = torch.tensor(name_embedding.values.tolist(), dtype=torch.float32)

            author_id = torch.tensor(author_id.values, dtype=torch.float32)

            total_time = torch.tensor(total_time.values, dtype=torch.float32)

            category_embedding = torch.tensor(category_embedding.values.tolist(), dtype=torch.float32)

            aggr_rating = torch.tensor(aggr_rating.values, dtype=torch.float32)

            review_count = torch.tensor(review_count.values, dtype=torch.float32)

            calories = torch.tensor(calories.values, dtype=torch.float32)

            fat_content = torch.tensor(fat_content.values, dtype=torch.float32)

            saturated_fat_content = torch.tensor(saturated_fat_content.values, dtype=torch.float32)

            cholesterol_content = torch.tensor(cholesterol_content.values, dtype=torch.float32)

            sodium_content = torch.tensor(sodium_content.values, dtype=torch.float32)

            carbohydrate_content = torch.tensor(carbohydrate_content.values, dtype=torch.float32)

            fiber_content = torch.tensor(fiber_content.values, dtype=torch.float32)

            sugar_content = torch.tensor(sugar_content.values, dtype=torch.float32)

            protein_content = torch.tensor(protein_content.values, dtype=torch.float32)
            
        # Concatenate all features
        x = torch.cat((
        recipe_id.unsqueeze(1),
        name_embedding, 
        author_id.unsqueeze(1), 
        total_time.unsqueeze(1), 
        category_embedding, 
        aggr_rating.unsqueeze(1), 
        review_count.unsqueeze(1), 
        calories.unsqueeze(1), 
        fat_content.unsqueeze(1), 
        saturated_fat_content.unsqueeze(1), 
        cholesterol_content.unsqueeze(1), 
        sodium_content.unsqueeze(1), 
        carbohydrate_content.unsqueeze(1), 
        fiber_content.unsqueeze(1), 
        sugar_content.unsqueeze(1), 
        protein_content.unsqueeze(1)
        ), dim=1)
    
        # Pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Final embedding vector
        return x


## Paired Dataset

In [195]:
class PairedDataset(torch.utils.data.Dataset):
    def __init__(self, review_id_to_embedding, recipe_id_to_embedding, review_to_recipe_map):
        """
        Create a paired dataset for positive review-recipe pairs.
        :param review_id_to_embedding: Dictionary mapping ReviewId to review embedding.
        :param recipe_id_to_embedding: Dictionary mapping RecipeId to recipe embedding.
        :param review_to_recipe_map: Dictionary mapping ReviewId to RecipeId.
        """
        self.review_id_to_embedding = review_id_to_embedding
        self.recipe_id_to_embedding = recipe_id_to_embedding
        self.review_to_recipe_map = review_to_recipe_map
        self.review_ids = list(self.review_to_recipe_map.keys())

    def __len__(self):
        return len(self.review_ids)

    def __getitem__(self, idx):
        if isinstance(idx, (list, torch.Tensor)):  # Handle batch indices
            return [self.__getitem__(i) for i in idx]
        # Get review embedding and ID
        
        review_id = self.review_ids[idx]
        review_embedding = self.review_id_to_embedding[review_id]

        # Find the associated RecipeId
        recipe_id = self.review_to_recipe_map[review_id]
        
        recipe_embedding = self.recipe_id_to_embedding[recipe_id]

        return review_id, recipe_id, review_embedding, recipe_embedding


In [ ]:
# 428602 229632
# print(valid_review_data[valid_review_data["reviewid"] == 428602])
# review_to_recipe_map[229632]

## Dataloaders

In [196]:
from torch.utils.data import DataLoader

batch_size = 32

# Create review and recipe dataloaders
print(train_review_data.shape)
# Create the dataset
review_dataset = ReviewsDataset(train_review_data)
recipe_dataset = RecipeDataset(train_recipe_data)

# Define dimensions
embedding_dim = 32  # Size of the output embedding vector (arbitrary, tunable)
review_embedding_dim = recipe_embedding_dim = 384  # Dimensionality of the "review" feature

# Initialize the model
review_tower = ReviewTower(review_embedding_dim=review_embedding_dim, embedding_dim=embedding_dim)
recipe_tower = RecipeTower(recipe_embedding_dim=recipe_embedding_dim, embedding_dim=embedding_dim)

# Create a DataLoader for batching
batch_size = 32
reviews_dataloader = DataLoader(review_dataset, batch_size=batch_size, shuffle=True)
recipe_dataloader = DataLoader(recipe_dataset, batch_size=batch_size, shuffle=True)

# Example: Iterate through the DataLoader
for batch in reviews_dataloader:
    print(type(batch["review_embedding"]))
    review_embeddings = review_tower(batch["review_embedding"], batch["month_sin"], batch["month_cos"], batch["year"], batch["rating"])
    print("Review Embedding shape:", review_embeddings.shape)
    print(review_embeddings)
    break

for batch in recipe_dataloader:
    recipe_embeddings = recipe_tower(
        batch["recipe_id"],
        batch["name_embedding"], 
        batch["author_id"], 
        batch["total_time"], 
        batch["category_embedding"], 
        batch["aggr_rating"],
        batch["review_count"], 
        batch["calories"], 
        batch["fat_content"], 
        batch["saturated_fat_content"], 
        batch["cholesterol_content"], 
        batch["sodium_content"], 
        batch["carbohydrate_content"], 
        batch["fiber_content"], 
        batch["sugar_content"], 
        batch["protein_content"]
    )
    print("Recipe Embedding shape:", recipe_embeddings.shape)
    print(recipe_embeddings)
    break

(397019, 8)
<class 'torch.Tensor'>
Review Embedding shape: torch.Size([32, 32])
tensor([[ 0.0704,  0.1354,  0.1063,  ...,  0.0256, -0.1120, -0.0522],
        [ 0.0729,  0.1377,  0.0919,  ...,  0.0289, -0.1111, -0.0554],
        [ 0.0851,  0.1474,  0.0790,  ...,  0.0126, -0.1227, -0.0411],
        ...,
        [ 0.0616,  0.1425,  0.1012,  ...,  0.0131, -0.1057, -0.0464],
        [ 0.0769,  0.1439,  0.1009,  ...,  0.0157, -0.1119, -0.0469],
        [ 0.0663,  0.1350,  0.1040,  ...,  0.0206, -0.1101, -0.0454]],
       grad_fn=<AddmmBackward0>)
Recipe Embedding shape: torch.Size([32, 32])
tensor([[-7599.8984,  3539.7825,  2886.6223,  ..., -2559.5818, -3824.5593,
          7678.2231],
        [  -28.7264,    15.2036,    14.0608,  ...,   -11.9686,   -13.8472,
            27.1912],
        [ -331.7814,   151.1247,   158.7859,  ...,  -127.4518,  -169.6227,
           315.4817],
        ...,
        [-6923.7520,  3054.8313,  2754.1338,  ..., -2485.4910, -3529.0249,
          6654.2417],
       

## Embeddings for Paired Dataset

In [192]:
review_embeddings = []
review_ids = []

# Generate embeddings for all reviews
review_tower.eval()
with torch.no_grad():
    for batch in reviews_dataloader:
        # Pass features through the ReviewTower
        embeddings = review_tower(
            batch["review_embedding"], batch["month_sin"], batch["month_cos"], 
            batch["year"], batch["rating"]
        )
        review_embeddings.append(embeddings)
        review_ids.extend(batch["review_id"])  # Assuming the batch includes ReviewId

# Combine all embeddings and IDs
review_embeddings = torch.cat(review_embeddings, dim=0)  # Shape: [num_reviews, embedding_dim]


In [53]:
review_id_to_embedding = {
    rid.item(): emb for rid, emb in zip(review_ids, review_embeddings)
}

In [191]:
recipe_embeddings = []
recipe_ids = []

# Generate embeddings for all recipes
recipe_tower.eval()
with torch.no_grad():
    for batch in recipe_dataloader:
        # Pass features through the RecipeTower
        embeddings = recipe_tower(
            batch["recipe_id"], batch["name_embedding"], batch["author_id"], 
            batch["total_time"], batch["category_embedding"], batch["aggr_rating"],
            batch["review_count"], batch["calories"], batch["fat_content"], 
            batch["saturated_fat_content"], batch["cholesterol_content"], 
            batch["sodium_content"], batch["carbohydrate_content"], batch["fiber_content"], 
            batch["sugar_content"], batch["protein_content"]
        )
        recipe_embeddings.append(embeddings)
        recipe_ids.extend(batch["recipe_id"])  # Assuming the batch includes RecipeId

# Combine all embeddings and IDs
recipe_embeddings = torch.cat(recipe_embeddings, dim=0)  # Shape: [num_recipes, embedding_dim]


In [54]:
# Generate dictionary mapping RecipeId to its embedding
recipe_id_to_embedding = {
    rid: emb for rid, emb in zip(recipe_ids, recipe_embeddings)
}

In [76]:
print(len(recipe_embeddings))

418013


In [88]:
print(recipe_ids[:10])
recipe_ids = [tensor.item() for tensor in recipe_ids]
print(recipe_ids[:10])

[284397, 27984, 237571, 292275, 513725, 534011, 142314, 376413, 359789, 55206]


AttributeError: 'int' object has no attribute 'item'

In [96]:
recipe_id_to_embedding = {int(k.item()): v for k, v in recipe_id_to_embedding.items()}

In [97]:
review_to_recipe_map = {
    review_id: recipe_id for review_id, recipe_id in zip(train_review_data["reviewid"], train_review_data["recipeid"])
    if recipe_id in recipe_id_to_embedding
}

In [95]:
# Filter out reviews without a matching RecipeId in the recipe dataset
valid_recipe_ids = set(recipe_ids) 
print(len(valid_recipe_ids))
train_review_data = train_review_data[train_review_data["recipeid"].isin(recipe_ids)]
print(train_review_data.shape)

418013
(397019, 8)


In [98]:
print(train_review_data["recipeid"].dtype)
print(type(next(iter(recipe_id_to_embedding.keys()))))

int64
<class 'int'>


## Create PairDataset

In [197]:
# Create paired dataset
print(len(review_id_to_embedding))
print(len(recipe_id_to_embedding))
print(len(review_to_recipe_map))


paired_dataset = PairedDataset(
    review_id_to_embedding=review_id_to_embedding,
    recipe_id_to_embedding=recipe_id_to_embedding,
    review_to_recipe_map=review_to_recipe_map
)

# Create DataLoader
paired_dataloader = DataLoader(paired_dataset, batch_size=batch_size, shuffle=True)

397019
418013
397019


## Two-Tower Model

In [108]:
class TwoTowerRecommender(nn.Module):
    def __init__(self, review_tower, recipe_tower, similarity_fn=cosine_similarity):
        super(TwoTowerRecommender, self).__init__()
        self.review_tower = review_tower  # Predefined Review Tower
        self.recipe_tower = recipe_tower  # Predefined Recipe Tower
        self.similarity_fn = similarity_fn

    def forward(self, review_rows, recipe_rows):
        # Extract review features (assuming these columns exist in `review_rows`)
        review_embedding = review_rows["review"]  # This is the precomputed review embedding
        month_sin = review_rows["month_sin"]
        month_cos = review_rows["month_cos"]
        year = review_rows["year"]
        rating = review_rows["rating"]

        # Pass review features to review_tower
        review_embeddings = self.review_tower(review_embedding, month_sin, month_cos, year, rating)

        # Extract recipe features (assuming these columns exist in `recipe_rows`)
        recipe_id = recipe_rows["recipeid"]
        name_embedding = recipe_rows["name"]
        author_id = recipe_rows["authorid"]
        total_time = recipe_rows["totaltime"]
        category_embedding = recipe_rows["recipecategory"]
        aggr_rating = recipe_rows["aggregatedrating"]
        review_count = recipe_rows["reviewcount"]
        calories = recipe_rows["calories"]
        fat_content = recipe_rows["fatcontent"]
        saturated_fat_content = recipe_rows["saturatedfatcontent"]
        cholesterol_content = recipe_rows["cholesterolcontent"]
        sodium_content = recipe_rows["sodiumcontent"]
        carbohydrate_content = recipe_rows["carbohydratecontent"]
        fiber_content = recipe_rows["fibercontent"]
        sugar_content = recipe_rows["sugarcontent"]
        protein_content = recipe_rows["proteincontent"]

        # Pass recipe features to recipe_tower
        item_embeddings = self.recipe_tower(
            recipe_id, name_embedding, author_id, total_time, category_embedding,
            aggr_rating, review_count, calories, fat_content, saturated_fat_content,
            cholesterol_content, sodium_content, carbohydrate_content, fiber_content,
            sugar_content, protein_content
        )

        # Compute similarity score
        scores = torch.matmul(review_embeddings, item_embeddings.T)  # Compute similarity
        return scores


# Training

In [106]:
from torch.nn.functional import cosine_similarity

# Define a contrastive loss function
def contrastive_loss(scores, labels, margin=1.0):
    positive_scores = scores[labels == 1]
    negative_scores = scores[labels == 0]
    loss = torch.clamp(margin - positive_scores + negative_scores, min=0).mean()
    return loss

In [207]:
def add_negatives(paired_dataset, review_ids, recipe_ids, num_negatives=1):
    """
    Generate positive and negative samples using the paired dataset.
    :param paired_dataset: The PairedDataset instance containing review and recipe embeddings.
    :param review_ids: Tensor containing review IDs for the batch.
    :param recipe_ids: Tensor containing corresponding recipe IDs for the batch.
    :param num_negatives: Number of negative samples to generate per positive.
    :return: Tensors for review embeddings, recipe embeddings, and labels.
    """
    
    pos_reviews = []
    pos_recipes = []
    
    # Loop through the batch of review and recipe IDs
    for review_id, recipe_id in zip(review_ids, recipe_ids):
        # Get the corresponding rows from the DataFrames
        review_row = train_review_data.loc[train_review_data["reviewid"] == review_id.item()].iloc[0]
        recipe_row = train_recipe_data.loc[train_recipe_data["recipeid"] == recipe_id.item()].iloc[0]
        
        # Extract features from the review and recipe rows
        review_features = (
            review_row["review"], 
            review_row["month_sin"], 
            review_row["month_cos"], 
            review_row["year"], 
            review_row["rating"]
        )
        
        recipe_features = (
            recipe_row["recipeid"], 
            recipe_row["name"], 
            recipe_row["authorid"], 
            recipe_row["totaltime"], 
            recipe_row["recipecategory"],
            recipe_row["aggregatedrating"], 
            recipe_row["reviewcount"], 
            recipe_row["calories"],
            recipe_row["fatcontent"], 
            recipe_row["saturatedfatcontent"], 
            recipe_row["cholesterolcontent"], 
            recipe_row["sodiumcontent"], 
            recipe_row["carbohydratecontent"], 
            recipe_row["fibercontent"], 
            recipe_row["sugarcontent"], 
            recipe_row["proteincontent"]
        )
        
        # Append to positive pairs
        pos_reviews.append(review_features)
        pos_recipes.append(recipe_features)
    
    # Convert to tensors
    pos_reviews = torch.stack(pos_reviews)  # Shape: [batch_size, review_feature_dim]
    pos_recipes = torch.stack(pos_recipes)  # Shape: [batch_size, recipe_feature_dim]

    # Negative sampling
    neg_reviews = []
    neg_recipes = []
    for _ in range(num_negatives):
        neg_recipe_indices = torch.randint(len(paired_dataset), (len(review_ids),))
        for idx in neg_recipe_indices:
            # Here we are selecting random negative recipes by idx
            neg_reviews.append(pos_reviews)  # Use the same reviews
            neg_recipe_row = paired_dataset[idx][1]  # Get the corresponding recipe row (embedding)
            neg_recipes.append(neg_recipe_row)
    
    # Stack negative tensors
    neg_reviews = torch.cat(neg_reviews, dim=0)
    neg_recipes = torch.stack(neg_recipes, dim=0)
    
    # Combine positive and negative samples
    all_reviews = torch.cat([pos_reviews, neg_reviews], dim=0)
    all_recipes = torch.cat([pos_recipes, neg_recipes], dim=0)
    all_labels = torch.cat([
        torch.ones(len(pos_reviews)),  # Positive labels
        torch.zeros(len(neg_recipes))  # Negative labels
    ], dim=0)

    return all_reviews, all_recipes, all_labels


## Training Loop

In [203]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [208]:
# Initialize the model
from tqdm import tqdm

model = TwoTowerRecommender(review_tower, recipe_tower, similarity_fn=cosine_similarity)

# Binary Cross-Entropy Loss
loss_fn = nn.BCEWithLogitsLoss()

# Define loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    model.train()
    running_loss = 0.0

    for review_id, recipe_id, _, _ in tqdm(paired_dataloader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        # Generate positive and negative samples
        print(review_id, recipe_id)
        review_features, recipe_features, labels = add_negatives(paired_dataset, review_id, recipe_id, num_negatives=1)

        # Move data to device (GPU or CPU)
        review_features = review_features.to(device)
        recipe_features = recipe_features.to(device)
        labels = labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Calculate the similarity for the positive and negative samples
        scores = model(review_features, recipe_features)

        # Compute contrastive loss
        labels = torch.arange(scores.size(0), dtype=torch.float32).to(scores.device)
        # Positive pairs on diagonal
        loss = loss_fn(scores, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(paired_dataloader)}")

tensor([  35009,  329825, 1227749,  170160, 2084073,  143703,  935137, 2011052,
        1052474,  263984,  825436,  887753, 2053805, 1295634,  976131,  224312,
         600001,  260203, 1184259, 1340003, 1021773,  247039,  748923,  707413,
         847922, 1213667,  119657, 1212479,  789633,  329777, 1479666,  425262]) tensor([ 35813, 123950, 234424, 107072, 205890, 106962, 227121, 301754, 145165,
        113861, 276268, 373619, 501408, 115565, 150770,  42007,  40681,  14106,
         33104,  88350, 385211,  95311,  78607,  69478, 245152, 423847,  10214,
        222188, 308090,  19479, 327922, 132816])


TypeError: expected Tensor as element 0 in argument 0, but got tuple

In [201]:
print(1245665 in train_review_data['reviewid'].values)
print(train_review_data[train_review_data['reviewid'] == 1245665])


True
        reviewid  recipeid  authorid    rating  \
436057   1245665     59148   1992585  0.465609   

                                                   review      year  \
436057  [-0.05396514, -0.06840967, -0.034714345, 0.068...  0.556577   

        month_sin  month_cos  
436057  -0.866025       -0.5  
